In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv("diabetes.csv")

In [3]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [4]:
df.corr()['Outcome']

Pregnancies                 0.221898
Glucose                     0.466581
BloodPressure               0.065068
SkinThickness               0.074752
Insulin                     0.130548
BMI                         0.292695
DiabetesPedigreeFunction    0.173844
Age                         0.238356
Outcome                     1.000000
Name: Outcome, dtype: float64

In [5]:
X = df.iloc[:,:-1].values
y = df.iloc[:,-1].values

In [6]:
from sklearn.preprocessing import StandardScaler

In [7]:
scaler = StandardScaler()

X = scaler.fit_transform(X)

In [8]:
X.shape

(768, 8)

In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=1)

In [10]:
import tensorflow 
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense

In [11]:
model = Sequential()

In [12]:
model.add(Dense(32, activation="relu", input_dim=8))
model.add(Dense(1, activation="sigmoid"))

model.compile(optimizer="adam", loss="binary_crossentropy", metrics=['accuracy'])

c:\Users\irbag\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [13]:
model.fit(X_train, y_train, batch_size=32, epochs=10)

Epoch 1/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5110 - loss: 0.7354  
Epoch 2/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5930 - loss: 0.6761 
Epoch 3/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7181 - loss: 0.6065 
Epoch 4/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7031 - loss: 0.5904 
Epoch 5/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7220 - loss: 0.5610 
Epoch 6/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7090 - loss: 0.5501 
Epoch 7/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7213 - loss: 0.5404 
Epoch 8/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7764 - loss: 0.5048 
Epoch 9/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7793 - loss: 0.4905 
Epoch 10/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7776 - loss: 0.4842 


In [14]:
import kerastuner as kt

C:\Users\irbag\AppData\Local\Temp\ipykernel_236476\1654478174.py:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  import kerastuner as kt


In [15]:
def build_model(hp):
  model = Sequential()

  model.add(Dense(32, activation="relu", input_dim=8))
  model.add(Dense(1, activation="sigmoid"))

  optimizer = hp.Choice('optimizer', values=['adam', 'rmsprop', 'sgd', 'adadelta'])

  model.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=['accuracy'])
  return model

In [16]:
tuner = kt.RandomSearch(
  build_model,
  objective='val_accuracy',
  max_trials=5
)

Reloading Tuner from .\untitled_project\tuner0.json


In [17]:
tuner.search(X_train,y_train, epochs=5, validation_data=(X_test,y_test))

In [18]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'adam'}

In [19]:
model = tuner.get_best_models(num_models=1)[0]

c:\Users\irbag\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\saving\saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [20]:
model.fit(X_train, y_train, batch_size=32, epochs=100, initial_epoch=6, validation_data=(X_test, y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.7309 - loss: 0.5258 - val_accuracy: 0.7922 - val_loss: 0.4988
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7628 - loss: 0.4801 - val_accuracy: 0.7922 - val_loss: 0.4900
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7646 - loss: 0.4956 - val_accuracy: 0.7857 - val_loss: 0.4850
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7829 - loss: 0.4557 - val_accuracy: 0.7792 - val_loss: 0.4802
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7937 - loss: 0.4528 - val_accuracy: 0.7792 - val_loss: 0.4767
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7582 - loss: 0.4732 - val_accuracy: 0.7857 - val_loss: 0.4751
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7956 - loss: 0.4668 - val_accuracy: 0.7857 - val_loss: 0.4736
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7701 - loss: 0.4528 - val_accuracy: 0.779

In [21]:
def build_model(hp):
  model = Sequential()

  units = hp.Int('units', 8,128)
  
  model.add(Dense(units=units, activation="relu", input_dim=8))
  model.add(Dense(1, activation="sigmoid"))

  model.compile(optimizer="adam", loss="binary_crossentropy", metrics=['accuracy'])
  return model


In [22]:
tuner = kt.RandomSearch(
  build_model,
  objective='val_accuracy',
  max_trials=5
)

Reloading Tuner from .\untitled_project\tuner0.json


In [23]:
tuner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

In [24]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'adam'}

In [25]:
# no of hidden layers

def build_model(hp):
  model= Sequential()

  model.add(Dense(72, activation="relu", input_dim=8))

  for i in range(hp.Int('num_layers', min_value=1, max_value=10)):
    model.add(Dense(72, activation="relu"))
  
  model.add(Dense(1, activation="sigmoid"))

  model.compile(optimizer="Adam", loss="binary_crossentropy", metrics=['accuracy'])
  
  return model

In [26]:
tuner = kt.RandomSearch(
  build_model,
  objective="val_accuracy",
  max_trials=5
)

Reloading Tuner from .\untitled_project\tuner0.json


In [27]:
tuner.search(X_train, y_train, epochs=50, validation_data=(X_test, y_test))

In [28]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'adam'}